In [1]:
#| default_exp ndwi_class

# NDWI 

In [2]:
#|hide
from nbdev.showdoc import *

In [3]:
#|export
import rasterio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

from flood_exercise import utils_func
from flood_exercise import const_vals as CONST

In [4]:
#|export

class ndwi():

  def __init__(self,
               path_to_s2_tiles : str , #path to the folder that contains the S2 images
               path_to_labeled_tiles : str , #path to the labled images (values -1,0,1)
               ):
    # get the list of tiles from each folder , get only files end with .tif
    self.tiles_s2 = utils_func.load_list_paths(path_to_s2_tiles, filter_file = True)
    self.labels = utils_func.load_list_paths(path_to_labeled_tiles , filter_file = True)


    # get only tiles from specific region (Bolivia for the task) 
    self.tiles_s2_for_evaluation = [x for x in self.tiles_s2 if CONST.REGION_STR_2 in x]

    #calculate threshold
    self.threshold = self._get_ndwi_threshold_()

    # iterate through sentinel-2 images , get % of water in label image and S2 image
    collector = {CONST.WATER_DF_TILE_ID_STR : [],
                 CONST.WATER_DF_WATER_PERC_STR : [],
                 CONST.WATER_DF_WATER_PERC_LABEL_STR : [],
                 CONST.WATER_DF_PATH_STR : []}
    for s2_path in self.tiles_s2:
        #find the matching label tile 
        path_id = s2_path.split(CONST.SPLIT_TILES_NAMES_STR1)[-1].split(CONST.SPLIT_TILES_NAMES_STR2)[1]
        match_label_tile_path = [x for x in self.labels if path_id in x][0]

        #calculate NDWI for S2 image 
        masked_ndwi,perc_water=self._water_prob_new_img_(path = s2_path)

        #calculate water percentage in label image
        label_water_perc = self._water_perc_label_img_(path = match_label_tile_path)



  def _ndwi_s2_(self ,
                     path):
         
         with rasterio.open(path) as src:
            green = src.read(CONST.GREEN_BAND)
            nir = src.read(CONST.NIR_BAND)

            ndwi = (green - nir) / (green + nir)

         return ndwi 
  


  def _get_ndwi_threshold_(self,
                           ):
    
       threshold = [None , None]

       #for each image , calculate NDWI ,get the match labled image
       for s2_path in self.tiles_s2_for_evaluation:
          
          #calculate NDWI, get the image id to match with the labels
          path_id = s2_path.split(CONST.SPLIT_TILES_NAMES_STR1)[-1].split(CONST.SPLIT_TILES_NAMES_STR2)[1]
          ndwi_img  = self._ndwi_s2_(s2_path)

          # match labeled image
          match_label_tile_path = [x for x in self.labels if path_id in x][0]
          match_label_tile = rasterio.open(match_label_tile_path).read(1)  

          #create water mask : change -1 to 0 
          water_mask = np.where(match_label_tile == -1, 0, match_label_tile)
          ndwi_img = np.where(ndwi_img == -1, np.NaN , ndwi_img)

          #multiply water_mask with NDWI :
          mask_ndwi = ndwi_img * water_mask

          # plt.imshow(mask_ndwi)
          # plt.title(path_id)
          # plt.show()
          
          #caluclate the min and max 
          min_val = np.nanmin(mask_ndwi)
          max_val = np.nanmax(mask_ndwi)


          if threshold[0] is None or min_val < threshold[0]:
              threshold[0] = min_val
          if threshold[1] is None or max_val > threshold[1]:
              threshold[1] = max_val

          # print(f' current threshold : {threshold} , path: {path_id}')


       return threshold



  def _water_prob_new_img_(self,
                           path : str , # path to raster image
                           ):

        #calcualte NDWI
        ndwi = self._ndwi_s2_(path)

        # mask the image to get only water pixels based on threshold
        mask_with_threshold_condition = (ndwi > self.threshold[0]) & (ndwi < 0)

        # masked_ndwi = np.where(mask_with_threshold_condition, 0 , 1)
        masked_ndwi = np.where(mask_with_threshold_condition, 0 , ndwi)
        masked_ndwi = np.nan_to_num(masked_ndwi, nan=0)
        masked_ndwi = np.where(masked_ndwi==0, 0 , 1)
        
        # plt.imshow(masked_ndwi)
        # plt.title(s2_path)
        # plt.colorbar()
        # plt.show()

        #calculate precentage of water pixel out of all the pixels in the image
        # perc_water = round((np.sum(masked_ndwi) / (masked_ndwi.shape[0]*masked_ndwi.shape[1]))*100,2)
        perc_water = round(( np.count_nonzero(masked_ndwi == CONST.WATER_VALUE) / (masked_ndwi.shape[0]*masked_ndwi.shape[1]))*100,2)

        
        return masked_ndwi,perc_water
  

  def _water_perc_label_img_(self,
                             path : str , # path to labeled image
                             ):
      
      with rasterio.open(path) as src:
          #read the image as numpy array
          arr = src.read()
          # printpercentage of pixels with water flag (value=1)
          perc_water = round(( np.count_nonzero(arr == CONST.WATER_VALUE) / (arr.shape[0]*arr.shape[1]))*100,2)

      return perc_water

     


In [5]:
instance = ndwi(path_to_s2_tiles = r"D:\git\flood_exercise\S2",
                path_to_labeled_tiles= r"D:\git\flood_exercise\S2_HANDLABELED")
instance.threshold
# instance.water_perc.to_csv(r'D:\git\flood_exercise\RESULTS\water_perc_Bolivia.csv')

C:\Users\USER\AppData\Local\Temp\ipykernel_18388\2130820773.py:31: RuntimeWarning: invalid value encountered in divide
  ndwi = (green - nir) / (green + nir)


AttributeError: 'ndwi' object has no attribute '_water_prob_new_imgs_'

In [ ]:
src = rasterio.open(r'D:\git\flood_exercise\S2_HANDLABELED\Bolivia_23014_LabelHand.tif')
arr = src.read()
np.count_nonzero(arr == -1)

In [ ]:
np.count_nonzero(arr == -1)+np.count_nonzero(arr == 0)+np.count_nonzero(arr == 1)

In [ ]:
arr.shape[2]*arr.shape[1]

In [ ]:
instance.test[:100,:100]

In [ ]:
instance.labels[0].split("\\")[-1].split('_')[1]

In [ ]:
#|export

class ndwi():

  def __init__(self,
      path_to_imgs : str , # path to the folder that contains the images
      ):
     
     # get list of tif files
     self.list_of_files = utils_func.load_list_paths(path_to_imgs,filter_file = True)
     self.list_of_files = [x for x in self.list_of_files if 'Bolivia' in x]
     
     collector = {CONST.IMG_PATH_WATER_STR : [] , CONST.WATER_PERC_STR : []}
     # calculate NDWI and NDWI mask per image
     for path in self.list_of_files:
         self.ndwi_img ,self.mask , self.perc_water = self._ndwi_s2_(path)
         collector[CONST.IMG_PATH_WATER_STR].append(path)
         collector[CONST.WATER_PERC_STR].append(self.perc_water)

     self.water_perc = pd.DataFrame(collector)
           
  def _ndwi_s2_(self ,
                     path):
         
         with rasterio.open(path) as src:
            green = src.read(CONST.GREEN_BAND)
            nir = src.read(CONST.NIR_BAND)

            ndwi = (green - nir) / (green + nir)

            # Replace NaN values with 0
            ndwi = np.nan_to_num(ndwi, nan=-1)
            # generate mask 
            mask = np.where(ndwi<0 ,0 ,1)

            #calculate precentage of water pixel out of all the pixels in the image
            # count_water_pixels = np.sum(mask)
            # total_pixels = mask.shape[0]*mask.shape[1]
            perc_water = round((np.sum(mask) / (mask.shape[0]*mask.shape[1]))*100,2)
         return ndwi , mask ,perc_water
     




In [ ]:
instance = ndwi(path_to_imgs = r"D:\git\flood_exercise\S2")
instance.water_perc.to_csv(r'D:\git\flood_exercise\RESULTS\water_perc_Bolivia.csv')